# Sentiment Analysis with BERT using HuggingFace lib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/smile-annotations-final.csv',names = ['id','text','category'])
df.set_index('id',inplace=True)

In [5]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [6]:
df.category.value_counts()  

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [7]:
df = df[-df.category.str.contains('\|')]

In [8]:
df = df[df.category != 'nocode']

In [9]:
df

,text,category
id,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy
...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy


In [10]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [11]:
possible_labels = df.category.unique()

In [12]:
label_dict = {}
for index, label in enumerate(possible_labels):
    label_dict[label] = index

In [13]:
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [14]:
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df.index.values,
df.label.values,
test_size=0.15,
random_state=10,
stratify=df.label.values)

In [17]:
df['data_type'] = ['not_set']*df.shape[0]

In [18]:
df.loc[X_train,'data_type'] = 'train'
df.loc[X_test,'data_type'] = 'test'

In [19]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 901kB 39.3MB/s 
     |████████████████████████████████| 3.3MB 37.5MB/s 


In [20]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [21]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [22]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type == 'test'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [23]:
encoded_data_train

{'input_ids': tensor([[  101, 16092,  3897,  ...,     0,     0,     0],
        [  101,  1030, 27034,  ...,     0,     0,     0],
        [  101,  1030, 10682,  ...,     0,     0,     0],
        ...,
        [  101, 11047,  1030,  ...,     0,     0,     0],
        [  101,  1030,  3680,  ...,     0,     0,     0],
        [  101,  1030,  2120,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [24]:
input_ids_train = encoded_data_train['input_ids']
attention_mask_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df['data_type'] == "train"].label.values)

In [25]:
input_ids_test = encoded_data_test['input_ids']
attention_mask_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df['data_type'] == "test"].label.values)

In [26]:
dataset_train = TensorDataset(input_ids_train,attention_mask_train,labels_train)
dataset_test = TensorDataset(input_ids_test,attention_mask_test,labels_test)

In [27]:
from transformers import BertForSequenceClassification

In [28]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
num_labels = len(label_dict),
output_attentions = False,
output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [29]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [30]:
batch_size = 32

dataloader_train = DataLoader(dataset_train,
sampler = RandomSampler(dataset_train),
batch_size = batch_size
)

dataloader_test = DataLoader(dataset_test,
sampler = RandomSampler(dataset_test),
batch_size = batch_size
)

In [31]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [32]:
optimizer = AdamW(

    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [33]:
epochs = 10
scheduler = get_linear_schedule_with_warmup(optimizer, 0, len(dataloader_train)*epochs)

In [34]:
import numpy as np
from sklearn.metrics import f1_score


In [35]:
def f1_score_func(preds,labels):
    pred_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,pred_flat,average='weighted')


In [36]:
def accuracy_per_class(preds,labels):
    label_dict_inverse = {v:k for k,v in label_dict.items()}

    pred_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = pred_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print(f'Class : {label_dict_inverse[label]}')
        print(f'Accuracy : {len(y_preds[y_preds == label])} / {len(y_true)}\n')

In [37]:
import random

seed_val = 10
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [39]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [40]:
for epoch in tqdm(range(1,epochs+1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train,
    desc = 'Epoch {:1d}'.format(epoch),
    leave = False,
    disable = False
    )

    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
    
        inputs = {
            'input_ids' : batch[0],
            'attention_mask' : batch[1],
            'labels' : batch[2]
        }

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})

    torch.save(model.state_dict(),f'BERT_ft_epoch{epoch}.model')
    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training Loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_test)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 score (weighted): {val_f1}')




Epoch 1
Training Loss: 1.2215066075325012
Validation loss: 0.8226185355867658
F1 score (weighted): 0.6656119824269878



Epoch 2
Training Loss: 0.7792842298746109
Validation loss: 0.7313617978777204
F1 score (weighted): 0.6656119824269878



Epoch 3
Training Loss: 0.6817976869642735
Validation loss: 0.6227590824876513
F1 score (weighted): 0.6858900510876996



Epoch 4
Training Loss: 0.5786669693887234
Validation loss: 0.5836501632417951
F1 score (weighted): 0.6871187234886155



Epoch 5
Training Loss: 0.4839193273335695
Validation loss: 0.5463120469025203
F1 score (weighted): 0.7576636894620457



Epoch 6
Training Loss: 0.4192599967122078
Validation loss: 0.49798342159816195
F1 score (weighted): 0.7901883643638893



Epoch 7
Training Loss: 0.3796076666563749
Validation loss: 0.48334294131823946
F1 score (weighted): 0.8021060366928043



Epoch 8
Training Loss: 0.3246631480753422
Validation loss: 0.48596384695598055
F1 score (weighted): 0.8197940196816994



Epoch 9
Training Loss: 0.30517613645642994
Validation loss: 0.48234811425209045
F1 score (weighted): 0.8143113801674988



Epoch 10
Training Loss: 0.2891252409666777
Validation loss: 0.4776874291045325
F1 score (weighted): 0.8220442626462235

